In [519]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [520]:
## deal with train part:

# fill in 0 where is missing value for GAP in the Orders
train = pd.read_csv("trainGAP.csv")
train['DATE'] = pd.to_datetime(train['DATE'])
DISTRICT_HASH = pd.DataFrame( {'DISTRICT_HASH': train['DISTRICT_HASH'].drop_duplicates(),
                              'key':1})
DATE = pd.DataFrame( {'DATE' : pd.date_range('20160101',periods = 21), 'key' : 1})
SLOTID = pd.DataFrame( { 'SLOTID': range(144),'key':1})
merged = pd.merge(DISTRICT_ID, DATE, on='key')
merged = pd.merge(merged, SLOTID, on='key')
len(merged['DATE'])
merged = merged[['DISTRICT_HASH','DATE','SLOTID']]
merged2 = pd.merge(merged,train, how='left', on = ['DISTRICT_HASH','DATE','SLOTID'])
merged2['GAP']= merged2['GAP'].fillna(0)
train = merged2
train = train [['DISTRICT_HASH', 'DATE', 'SLOTID', 'GAP']]
del merged, merged2



In [521]:
#fill in the nearest-time value 
Weather = pd.read_csv('weather.csv')
Weather["WEATHER"] = Weather["WEATHER"].astype("category")
Weather['DATE'] = pd.to_datetime(Weather['DATE'])
Weahter = Weather.sort(columns = ['DATE','SLOTID'])
DATE = pd.DataFrame( {'DATE' : pd.date_range('20160101',periods = 21), 'key' : 1})
SLOTID = pd.DataFrame( { 'SLOTID': range(144),'key':1})
merged = pd.merge(DATE,SLOTID, on='key')
merged = merged[['DATE','SLOTID']]
merged2 = pd.merge(merged,Weather, how = 'left',on= ['DATE','SLOTID'])

isMissing = pd.isnull(merged2['WEATHER'])
first = 0
if isMissing[first]:
    first = first + 1 
for i in range(first):
    merged2.set_value(1,'WEATHER',merged2.iloc[first,]['WEATHER'])
    merged2.set_value(1,'TEMPERATURE',merged2.iloc[first,]['TEMPERATURE'])
    merged2.set_value(1,'PM25',merged2.iloc[first,]['PM25'])
for i in range(len(merged2['DATE'])):
    if isMissing[i]:
        merged2.set_value(i,'WEATHER',row['WEATHER'])
        merged2.set_value(i,'TEMPERATURE',row['TEMPERATURE'])
        merged2.set_value(i,'PM25', row['PM25'])
Weather = merged2
del merged, merged2
isMissing = pd.isnull(weather['WEATHER'])
isMissing.sum()

0

In [522]:
POI2 = pd.read_csv('POI2.csv')
DistMapping = pd.read_csv('DistMapping.csv')
Traffic = pd.read_csv("Traffic.csv")
Traffic['DATE'] = pd.to_datetime(Traffic['DATE'])
Traffic.columns

# fill the missing of the traffic
DISTRICT_ID = pd.DataFrame( {'DISTRICT_HASH': train['DISTRICT_HASH'].drop_duplicates(),
                              'key':1})
DATE = pd.DataFrame( {'DATE' : pd.date_range('20160101',periods = 21), 'key' : 1})
SLOTID = pd.DataFrame( { 'SLOTID': range(144),'key':1})
merged = pd.merge(DISTRICT_HASH, DATE, on='key')
merged = pd.merge(merged, SLOTID, on='key')
merged = merged[['DISTRICT_HASH','DATE','SLOTID']]
merged2= pd.merge(merged, Traffic, how ='left', on =['DISTRICT_HASH','DATE','SLOTID'] )
len(merged2['DATE'])

isMissing = pd.isnull(merged2['TJ_LEVEL1'])
first = 0
if isMissing[first]:
    first = first + 1 
merged2.set_value(1,'TJ_LEVEL1',merged2.iloc[first,]['TJ_LEVEL1'])
merged2.set_value(1,'TJ_LEVEL2',merged2.iloc[first,]['TJ_LEVEL2'])
merged2.set_value(1,'TJ_LEVEL3',merged2.iloc[first,]['TJ_LEVEL3'])
merged2.set_value(1,'TJ_LEVEL4',merged2.iloc[first,]['TJ_LEVEL4'])  
merged2.set_value(1,'TJ_TIME',merged2.iloc[first,]['TJ_TIME']) 
for i in range(len(merged2['DATE'])):
    if isMissing[i]:
        merged2.set_value(i,'TJ_LEVEL1',merged2.iloc[i-1,]['TJ_LEVEL1'])
        merged2.set_value(i,'TJ_LEVEL2',merged2.iloc[i-1,]['TJ_LEVEL2'])
        merged2.set_value(i,'TJ_LEVEL3',merged2.iloc[i-1,]['TJ_LEVEL3'])
        merged2.set_value(i,'TJ_LEVEL4',merged2.iloc[i-1,]['TJ_LEVEL4'])  
        merged2.set_value(i,'TJ_TIME',merged2.iloc[i-1,]['TJ_TIME'])
Traffic = merged2
del merged, merged2
isMissing.sum()

6031

In [523]:
#train, join tall the tables
train1 = pd.merge(train, Weather, how = 'left', on = ['DATE','SLOTID'])
train1 = pd.merge(train1,POI2, how = 'left', on = 'DISTRICT_HASH')
train1 = pd.merge(DistMapping, train1, on = 'DISTRICT_HASH')
train1 = pd.merge( train1, Traffic, how = 'left' ,on = ['DISTRICT_HASH','DATE','SLOTID'])
train1 = train1.sort(columns = ['ID','DATE','SLOTID'])
train1.to_csv("trainSet_nonMissing.csv")

In [524]:

#creat the test  set 

test = pd.read_csv('testGAP.csv')
test['DATE'] = pd.to_datetime(test['DATE'])
test.sort(columns= ['DISTRICT_HASH','DATE','SLOTID'])
DISTRICT_HASH_te = pd.DataFrame( {'DISTRICT_HASH': test['DISTRICT_HASH'].drop_duplicates(),
                              'key':1})
DATE_te = pd.DataFrame( {'DATE' : test['DATE'].drop_duplicates() , 'key' : 1})
SLOTID_te = pd.DataFrame( { 'SLOTID': test['SLOTID'].drop_duplicates(),'key':1})
merged = pd.merge(DISTRICT_HASH_te, DATE_te, on='key')
merged = pd.merge(merged, SLOTID_te, on='key')

merged = merged[['DISTRICT_HASH','DATE','SLOTID']]
merged2 = pd.merge(merged,test, how='left', on = ['DISTRICT_HASH','DATE','SLOTID'])
merged2['GAP']= merged2['GAP'].fillna(0)
test = merged2
test = test [['DISTRICT_HASH', 'DATE', 'SLOTID', 'GAP']]
del merged, merged2


In [525]:
#fill in missing values in weather  

Weather_te = pd.read_csv('weather_test.csv')
Weather_te["WEATHER"] = Weather_te["WEATHER"].astype("category")
Weather_te['DATE'] = pd.to_datetime(Weather_te['DATE'])
Weahter_te = Weather.sort(columns = ['DATE','SLOTID'])
DATE_te = pd.DataFrame( {'DATE' : Weather_te['DATE'].drop_duplicates() , 'key' : 1})
SLOTID_te = pd.DataFrame( { 'SLOTID': Weather_te['SLOTID'].drop_duplicates(),'key':1})
len(SLOTID_te)

merged = pd.merge(DATE_te,SLOTID_te, on='key')
merged = merged[['DATE','SLOTID']]
merged2 = pd.merge(merged,Weather_te, how = 'left',on= ['DATE','SLOTID'])

isMissing = pd.isnull(merged2['WEATHER'])
first = 0
if isMissing[first]:
    first = first + 1 
for i in range(first):
    merged2.set_value(1,'WEATHER',merged2.iloc[first,]['WEATHER'])
    merged2.set_value(1,'TEMPERATURE',merged2.iloc[first,]['TEMPERATURE'])
    merged2.set_value(1,'PM25',merged2.iloc[first,]['PM25'])
for i in range(len(merged2['DATE'])):
    if isMissing[i]:
        merged2.set_value(i,'WEATHER',row['WEATHER'])
        merged2.set_value(i,'TEMPERATURE',row['TEMPERATURE'])
        merged2.set_value(i,'PM25', row['PM25'])
Weather_te = merged2
del merged, merged2
isMissing = pd.isnull(Weather_te['WEATHER'])
isMissing.sum()

0

In [526]:
POI2_te = pd.read_csv('POI2_test.csv')
Traffic_te = pd.read_csv('traffic_test.csv')
Traffic_te['DATE'] = pd.to_datetime(Traffic_te['DATE'])
Traffic_te.columns

# fill the missing of the traffic
DISTRICT_ID = pd.DataFrame( {'DISTRICT_HASH': test['DISTRICT_HASH'].drop_duplicates(),
                              'key':1})
merged = pd.merge(DISTRICT_HASH_te, DATE_te, on='key')
merged = pd.merge(merged, SLOTID_te, on='key')
merged = merged[['DISTRICT_HASH','DATE','SLOTID']]
merged2= pd.merge(merged, Traffic_te, how ='left', on =['DISTRICT_HASH','DATE','SLOTID'] )
len(merged2['DATE'])

isMissing = pd.isnull(merged2['TJ_LEVEL1'])
first = 0
if isMissing[first]:
    first = first + 1 
merged2.set_value(1,'TJ_LEVEL1',merged2.iloc[first,]['TJ_LEVEL1'])
merged2.set_value(1,'TJ_LEVEL2',merged2.iloc[first,]['TJ_LEVEL2'])
merged2.set_value(1,'TJ_LEVEL3',merged2.iloc[first,]['TJ_LEVEL3'])
merged2.set_value(1,'TJ_LEVEL4',merged2.iloc[first,]['TJ_LEVEL4'])  
merged2.set_value(1,'TJ_TIME',merged2.iloc[first,]['TJ_TIME']) 
for i in range(len(merged2['DATE'])):
    if isMissing[i]:
        merged2.set_value(i,'TJ_LEVEL1',merged2.iloc[i-1,]['TJ_LEVEL1'])
        merged2.set_value(i,'TJ_LEVEL2',merged2.iloc[i-1,]['TJ_LEVEL2'])
        merged2.set_value(i,'TJ_LEVEL3',merged2.iloc[i-1,]['TJ_LEVEL3'])
        merged2.set_value(i,'TJ_LEVEL4',merged2.iloc[i-1,]['TJ_LEVEL4'])  
        merged2.set_value(i,'TJ_TIME',merged2.iloc[i-1,]['TJ_TIME'])
Traffic_te = merged2
del merged, merged2
isMissing = pd.isnull(Traffic_te['TJ_LEVEL1'])
isMissing.sum()

0

In [527]:
test1 = pd.merge(test, Weather_te, how = 'left', on = ['DATE','SLOTID'])
test1 = pd.merge(test1,POI2_te, how = 'left', on = 'DISTRICT_HASH')
test1 = pd.merge(DistMapping, test1, on = 'DISTRICT_HASH')
test1 = pd.merge( test1, Traffic_te, how = 'left' ,on = ['DISTRICT_HASH','DATE','SLOTID'])
test1 = test1.sort(columns = ['ID','DATE','SLOTID'])
test1.to_csv('testSet_nonMissing.csv')
len(test1['DATE'])

8910